In [1]:
import pandas as pd
import datasets
from bs4 import BeautifulSoup
from io import StringIO
import csv
import torch
import difflib
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    AutoTokenizer,
    GenerationConfig,
    TextStreamer,
    pipeline,
)

from tqdm import tqdm 
import pandas as pd
import re

/mnt/gime-extract/miniconda3/envs/llmenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/gime-extract/miniconda3/envs/llmenv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/mnt/gime-extract/miniconda3/envs/llmenv/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/mnt/gime-extract/miniconda3/envs/llmenv/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. 

In [2]:
data = datasets.load_dataset('FinGPT/fingpt-convfinqa')

In [3]:
train_dataset = data['train']
eval_dataset = data['test']

In [ ]:
html_content = train_dataset[0]

In [14]:
print(html_content)

{'input': "26 | 2009 annual report in fiscal 2008 , revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007 . all revenue components within the segment experienced growth during fiscal 2008 . license revenue generated the largest dollar growth in revenue as episys ae , our flagship core processing system aimed at larger credit unions , experienced strong sales throughout the year . support and service revenue , which is the largest component of total revenues for the credit union segment , experienced 34 percent growth in eft support and 10 percent growth in in-house support . gross profit in this business segment increased $ 9344 in fiscal 2008 compared to fiscal 2007 , due primarily to the increase in license revenue , which carries the highest margins . liquidity and capital resources we have historically generated positive cash flow from operations and have generally used funds generated from operations and short-term borrowings on

In [23]:
text = html_content['input']
print(text)

26 | 2009 annual report in fiscal 2008 , revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007 . all revenue components within the segment experienced growth during fiscal 2008 . license revenue generated the largest dollar growth in revenue as episys ae , our flagship core processing system aimed at larger credit unions , experienced strong sales throughout the year . support and service revenue , which is the largest component of total revenues for the credit union segment , experienced 34 percent growth in eft support and 10 percent growth in in-house support . gross profit in this business segment increased $ 9344 in fiscal 2008 compared to fiscal 2007 , due primarily to the increase in license revenue , which carries the highest margins . liquidity and capital resources we have historically generated positive cash flow from operations and have generally used funds generated from operations and short-term borrowings on our revolv

In [24]:
# Split text before and after the table
text_parts = text.split("the following table summarizes net cash from operating activities in the statement of cash flows :")

# Print the parts
for part in text_parts:
    print(part.strip())
    print('***')

26 | 2009 annual report in fiscal 2008 , revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007 . all revenue components within the segment experienced growth during fiscal 2008 . license revenue generated the largest dollar growth in revenue as episys ae , our flagship core processing system aimed at larger credit unions , experienced strong sales throughout the year . support and service revenue , which is the largest component of total revenues for the credit union segment , experienced 34 percent growth in eft support and 10 percent growth in in-house support . gross profit in this business segment increased $ 9344 in fiscal 2008 compared to fiscal 2007 , due primarily to the increase in license revenue , which carries the highest margins . liquidity and capital resources we have historically generated positive cash flow from operations and have generally used funds generated from operations and short-term borrowings on our revolv

In [15]:
soup = BeautifulSoup(html_content['input'], 'html.parser')

# Find the table
table = soup.find('table')

print(table)

<table class="wikitable"><tr><td>1</td><td>2008</td><td>year ended june 30 2009 2008</td><td>year ended june 30 2009 2008</td><td>year ended june 30 2009</td></tr><tr><td>2</td><td>net income</td><td>$ 103102</td><td>$ 104222</td><td>$ 104681</td></tr><tr><td>3</td><td>non-cash expenses</td><td>74397</td><td>70420</td><td>56348</td></tr><tr><td>4</td><td>change in receivables</td><td>21214</td><td>-2913 ( 2913 )</td><td>-28853 ( 28853 )</td></tr><tr><td>5</td><td>change in deferred revenue</td><td>21943</td><td>5100</td><td>24576</td></tr><tr><td>6</td><td>change in other assets and liabilities</td><td>-14068 ( 14068 )</td><td>4172</td><td>17495</td></tr><tr><td>7</td><td>net cash from operating activities</td><td>$ 206588</td><td>$ 181001</td><td>$ 174247</td></tr></table>


In [22]:
soup.get_text()

'26 | 2009 annual report in fiscal 2008 , revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007 . all revenue components within the segment experienced growth during fiscal 2008 . license revenue generated the largest dollar growth in revenue as episys ae , our flagship core processing system aimed at larger credit unions , experienced strong sales throughout the year . support and service revenue , which is the largest component of total revenues for the credit union segment , experienced 34 percent growth in eft support and 10 percent growth in in-house support . gross profit in this business segment increased $ 9344 in fiscal 2008 compared to fiscal 2007 , due primarily to the increase in license revenue , which carries the highest margins . liquidity and capital resources we have historically generated positive cash flow from operations and have generally used funds generated from operations and short-term borrowings on our revol

In [21]:
# Open StringIO object for writing
csv_buffer = StringIO()

# Write rows to the StringIO object
writer = csv.writer(csv_buffer)
rows = table.find_all('tr')
for row in rows:
    writer.writerow([cell.get_text() for cell in row.find_all(['td', 'th'])])

# Get the CSV data as a string
csv_data = csv_buffer.getvalue()

# Print CSV data (for demonstration)
print(csv_data)

1,2008,year ended june 30 2009 2008,year ended june 30 2009 2008,year ended june 30 2009
2,net income,$ 103102,$ 104222,$ 104681
3,non-cash expenses,74397,70420,56348
4,change in receivables,21214,-2913 ( 2913 ),-28853 ( 28853 )
5,change in deferred revenue,21943,5100,24576
6,change in other assets and liabilities,-14068 ( 14068 ),4172,17495
7,net cash from operating activities,$ 206588,$ 181001,$ 174247



In [4]:
model_id = 'RUCKBReasoning/TableLLM-13b'

In [25]:
ip = """[INST]Below are the first few lines of a CSV file. You need to write a Python program to solve the provided question.

Header and first few lines of CSV file:
{csv_data}

Question: what is the net cash from operating activities in 2009?
[/INST]
"""

In [27]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json: 100%|██████████| 872/872 [00:00<00:00, 1.84MB/s]
tokenizer.model: 100%|██████████| 500k/500k [00:00<00:00, 3.12MB/s]
tokenizer.json: 100%|██████████| 1.84M/1.84M [00:00<00:00, 1.95MB/s]
special_tokens_map.json: 100%|██████████| 435/435 [00:00<00:00, 1.95MB/s]


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto", torch_dtype=torch.float16,
)
generation_config = GenerationConfig.from_pretrained(model_id)
print(generation_config)

generation_config.max_new_tokens = 512
generation_config.temperature = 0.0001
generation_config.do_sample = True
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)


In [29]:
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    streamer=streamer,
)

In [ ]:
output = llm(ip)[0]['generated_text']
print(output)